## Проверка данных Цифромеда

In [2]:
import pandas as pd
import numpy as np
import os
import random

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")
# пути и списки файлов
# файлы берём непосредственно из хранилища нового бота посещений
e_path = "C:/work/new_bot/new/e"
s_path = "C:/work/new_bot/new/s"
dm_path = "C:/work/new_bot/new/dm"
e_file_list = os.listdir(e_path)
s_file_list = os.listdir(s_path)
dm_file_list = os.listdir(dm_path + "/" + max(os.listdir(dm_path)))

# типы ошибок
t_error_list = [
    "Ошибка валидации данных пациента",
    "Ошибка «РМИС не ответил»",
    "Внутренняя ошибка РМИС при создании записи",
    "Сервис МИС приостановлен",
    "Ошибка валидации схемы",
    "Время уже занято другим пациентом",
    # "Пациент уже записан к специалисту в этот день",
    "Услуга посмертно не оказывается",
    # "Пациент уже записан на это время к другому специалисту",
    # "Выбранное время доступно только для записи пациентов определенного возраста",
    "Время начала приема уже прошло",
]
tdb_error_list = [
    "Ошибка валидации данных пациента",
    "Ошибка «РМИС не ответил»",
    "Внутренняя ошибка РМИС при создании записи",
    "Сервис МИС приостановлен",
    "Ошибка валидации схемы",
    # "Время уже занято другим пациентом",
    # "Услуга посмертно не оказывается",
    # "Время начала приема уже прошло",
]
b_list = [
    "Пациент уже записан на это время к другому специалисту",
    "Пациент уже записан к специалисту в этот день",
    "Выбранное время доступно только для записи пациентов определенного возраста",
]
o_error_list = [
    "Значение не найдено",
]
r_error_list = [
    "Ошибка «СМЭВ-timeout»",
    "Внутренняя ошибка концентратора при создании записи",
    "Ошибка во входных параметрах при записи на прием",
]
any_error_list = [
    "Ошибка доставки формы при записи на прием",
    "Истекло время сессии пользователя",
    "Ошибка соответствия ответа запросу МИС",
]
draft_list = ["Ошибка отсутсвует"]
# словарь списков типов ошибок
e_type_dict = {
    "t": t_error_list,
    "tdb": tdb_error_list,
    "b": b_list,
    "o": o_error_list,
    "r": r_error_list,
    "any": any_error_list,
    "draft": draft_list,
}

In [2]:
# for file_name in e_file_list:
#     print(file_name)
#     df_tmp = pd.read_csv(e_path + "/" + file_name, sep=";")
#     break

01.11.csv


In [3]:
for file_name in e_file_list:
    df_tmp = pd.read_csv(e_path + "/" + file_name, sep=";")
    print(f"\nФайл {file_name}:")
    print(f"\t\tвсего строк: {df_tmp.shape[0]}")
    cnt_c = "верно" if df_tmp.shape[1] == 13 else "ошибка"
    print(f"\t\tвсего столбцов: {df_tmp.shape[1]} ({cnt_c})")
    print("\tПропуски в столбцах, где их быть не должно:")
    df_nonan = df_tmp[
        [
            "session_id",
            "Название субъекта РФ",
            "step_id",
            "step_name",
            "error_code",
            "final_state",
            "create_ts",
            "Тип записи",
            "referral_flg",
        ]
    ]
    n = 0
    for column in df_nonan.columns:
        if df_nonan[column].isnull().sum() != 0:
            print(
                f"\t\tстолбец '{column}':\t\t {df_nonan[column].isnull().sum()} ({round(df_nonan[column].isnull().sum()*100/df_nonan.shape[0], 2)} %)"
            )
            n += 1
    if n == 0:
        print("\t\tотсутствуют")
    print(
        "\tОшибки данных (количество и доля от числа строк на соответствующем этапе):"
    )
    # число строк на соответствующих этапах
    len_ValidatePerson = df_tmp[df_tmp["step_name"] == "ValidatePerson"].shape[0]
    len_MO = df_tmp[df_tmp["step_name"] == "MO"].shape[0]
    len_ServiceOrSpecs = df_tmp[df_tmp["step_name"] == "ServiceOrSpecs"].shape[0]
    len_Slot = df_tmp[df_tmp["step_name"] == "Slot"].shape[0]
    len_Book = df_tmp[df_tmp["step_name"] == "Book"].shape[0]

    """
    На этапе валидации пациента
    есть данные oid МО
    ИЛИ есть данные о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_ValidatePerson = df_tmp[
        (df_tmp["step_name"] == "ValidatePerson")
        & (
            df_tmp["mo_oid"].notnull()
            | df_tmp["mo_name"].notnull()
            | df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """
    На этапе выбора МО
    есть данные oid МО
    ИЛИ есть данные о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_MO = df_tmp[
        (df_tmp["step_name"] == "MO")
        & (
            df_tmp["mo_oid"].notnull()
            | df_tmp["mo_name"].notnull()
            | df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """
    На этапе выбора специальности врача
    нет данных об oid МО
    ИЛИ нет данных о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_ServiceOrSpecs = df_tmp[
        (df_tmp["step_name"] == "ServiceOrSpecs")
        & (
            ~df_tmp["mo_oid"].notnull()
            | ~df_tmp["mo_name"].notnull()
            | df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """"
    На этапе выбора слота
    нет данных об oid МО
    ИЛИ нет данных о названии МО
    ИЛИ нет данных о специальтности врача
    ИЛИ есть данные о времени слота
    """
    df_Slot = df_tmp[
        (df_tmp["step_name"] == "Slot")
        & (
            ~df_tmp["mo_oid"].notnull()
            | ~df_tmp["mo_name"].notnull()
            | ~df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """
    На этапе записи
    нет любых данных (ИЛИ)
    """
    df_Book = df_tmp[
        (df_tmp["step_name"] == "Book")
        & (
            ~df_tmp["mo_oid"].notnull()
            | ~df_tmp["mo_name"].notnull()
            | ~df_tmp["Post_name"].notnull()
            | ~df_tmp["slot_ts"].notnull()
        )
    ]

    if df_ValidatePerson.shape[0] >= 0:
        print(
            f"\t\tValidatePerson_uncorrected:\t{df_ValidatePerson.shape[0]}\t({round(df_ValidatePerson.shape[0]*100/len_ValidatePerson, 2)} %)"
        )
    if df_MO.shape[0] >= 0:
        print(
            f"\t\tMO_uncorrected:\t\t\t{df_MO.shape[0]}\t({round(df_MO.shape[0]*100/len_MO, 2)} %)"
        )
    if df_ServiceOrSpecs.shape[0] >= 0:
        print(
            f"\t\tServiceOrSpecs_uncorrected:\t{df_ServiceOrSpecs.shape[0]}\t({round(df_ServiceOrSpecs.shape[0]*100/len_ServiceOrSpecs, 2)} %)"
        )
    if df_Slot.shape[0] >= 0:
        print(
            f"\t\tSlot_uncorrected:\t\t{df_Slot.shape[0]}\t({round(df_Slot.shape[0]*100/len_Slot, 2)} %)"
        )
    if df_Book.shape[0] >= 0:
        print(
            f"\t\tBook_uncorrected:\t\t{df_Book.shape[0]}\t({round(df_Book.shape[0]*100/len_Book, 2)} %)"
        )
    if (
        df_ValidatePerson.shape[0]
        + df_MO.shape[0]
        + df_ServiceOrSpecs.shape[0]
        + df_Slot.shape[0]
        + df_Book.shape[0]
        == 0
    ):
        print("\t\tотсутствуют")

    # субъекты, которых нет на дашборде
    df_nosub = df_tmp[
        (df_tmp["Название субъекта РФ"] == "Федеральное медико-биологическое агентство")
        | (df_tmp["Название субъекта РФ"] == "г. Байконур")
    ][["Название субъекта РФ", "final_state"]]
    df_nosub["value"] = 1
    # ошибки, которых нет на дашборде
    df_nosub_e = (
        df_nosub.groupby(["Название субъекта РФ", "final_state"])
        .sum()
        .reset_index()[["final_state", "value"]]
    )
    no_db_e_dict = {}
    for key, value in e_type_dict.items():
        no_db_e_dict[str(key + "_ndb")] = df_nosub_e[
            df_nosub_e["final_state"].isin(value)
        ]["value"].sum()

    df_errors = pd.DataFrame(
        df_tmp[df_tmp["Тип записи"] == 3].groupby("final_state")["session_id"].count()
    ).reset_index()
    print('\tКоличество ошибок записи на приём (тип записи "К врачу"; рассылка (дашборд)):')
    o_e = df_errors[df_errors["final_state"].isin(o_error_list)]["session_id"].sum()
    print(f'\t\tорганизационные ошибки:\t\t{o_e}\t({o_e - no_db_e_dict["o_ndb"]})')
    t_e = df_errors[df_errors["final_state"].isin(t_error_list)]["session_id"].sum()
    t_e_db = (
        df_errors[df_errors["final_state"].isin(tdb_error_list)]["session_id"].sum()
        - no_db_e_dict["t_ndb"]
    )
    print(f"\t\tтехнические ошибки:\t\t{t_e}\t({t_e_db})")
    r_e = df_errors[df_errors["final_state"].isin(r_error_list)]["session_id"].sum()
    print(f'\t\tошибки ФЭР:\t\t\t{r_e}\t({r_e - no_db_e_dict["r_ndb"]})')
    any_e = df_errors[df_errors["final_state"].isin(any_error_list)]["session_id"].sum()
    print(f'\t\tпрочие ошибки:\t\t\t{any_e}\t({any_e - no_db_e_dict["any_ndb"]})')
    draft = df_errors[df_errors["final_state"].isin(draft_list)]["session_id"].sum()
    print(f'\t\tчерновики:\t\t\t{draft}\t({draft - no_db_e_dict["draft_ndb"]})')
    b = df_errors[df_errors["final_state"].isin(b_list)]["session_id"].sum()
    print(f'\t\tбизнес-правила:\t\t\t{b}\t({b - no_db_e_dict["b_ndb"]})')
    print("\tСубъекты, не отображаемые на дашборде:")
    if df_nosub.empty:
        print("\t\tотсутствуют")
    else:
        print(
            df_nosub.groupby(["Название субъекта РФ", "final_state"])
            .sum()
            .reset_index()
            .to_string(index=False)
        )
    # break


Файл 01.11.csv:
		всего строк: 524928
		всего столбцов: 13 (верно)
	Пропуски в столбцах, где их быть не должно:
		отсутствуют
	Ошибки данных (количество и доля от числа строк на соответствующем этапе):
		ValidatePerson_uncorrected:	4	(0.01 %)
		MO_uncorrected:			0	(0.0 %)
		ServiceOrSpecs_uncorrected:	3620	(3.72 %)
		Slot_uncorrected:		2646	(2.38 %)
		Book_uncorrected:		40	(0.22 %)
	Количество ошибок записи на приём (тип записи "К врачу"; рассылка (дашборд)):
		организационные ошибки:		68658	(68658)
		технические ошибки:		53821	(53821)
		ошибки ФЭР:			27113	(27113)
		прочие ошибки:			0	(0)
		черновики:			374375	(374375)
		бизнес-правила:			0	(0)
	Субъекты, не отображаемые на дашборде:
		отсутствуют

Файл 02.11.csv:
		всего строк: 496113
		всего столбцов: 13 (верно)
	Пропуски в столбцах, где их быть не должно:
		столбец 'Название субъекта РФ':		 1 (0.0 %)
	Ошибки данных (количество и доля от числа строк на соответствующем этапе):
		ValidatePerson_uncorrected:	1	(0.0 %)
		MO_uncorrected

In [4]:
for file_name in s_file_list:
    df_tmp_s = pd.read_csv(s_path + "/" + file_name, sep=";")
    print(f"\nФайл {file_name}:")
    break



Файл И38_успешные_сессии_20231101.csv:


In [50]:
print(f"\nФайл {file_name}:")
print(f"\t\tвсего строк: {df_tmp_s.shape[0]}")
cnt_s = "верно" if df_tmp_s.shape[1] == 9 else "ошибка"
print(f"\t\tвсего столбцов: {df_tmp_s.shape[1]} ({cnt_s})")
print("\tПропуски данных (всего (доля от общего числа строк)):")
# пропуски данных
for column in df_tmp_s.columns:
    print(
        f"\t\t{column}:\t\t{df_tmp_s[column].isnull().sum()} ({round(df_tmp_s[column].isnull().sum()*100/df_tmp_s.shape[0], 2)} %)"
    )
# субъекты, которых нет на дашборде - не доделано
df_nosub_s = df_tmp_s[
    (df_tmp_s["Название субъекта РФ"] == "Федеральное медико-биологическое агентство")
    | (df_tmp_s["Название субъекта РФ"] == "г. Байконур")
]
df_nosub_s["value"] = 1


Файл И38_успешные_сессии_20231101.csv:
		всего строк: 130303
		всего столбцов: 9 (верно)
	Пропуски данных (всего (доля от общего числа строк)):
		Дата:		0 (0.0 %)
		session_id:		0 (0.0 %)
		Название субъекта РФ:		0 (0.0 %)
		sp_oid:		2075 (1.59 %)
		sp_name:		0 (0.0 %)
		post_name:		0 (0.0 %)
		slot_ts:		0 (0.0 %)
		create_ts:		0 (0.0 %)
		Тип записи:		0 (0.0 %)


In [48]:
df_tmp_s[df_tmp_s["Тип записи"] == 3]

,Дата,session_id,Название субъекта РФ,sp_oid,sp_name,post_name,slot_ts,create_ts,Тип записи
0,2023-11-01,3454575824,Забайкальский край,1.2.643.5.1.13.13.12.2.75.7725.0.18149,"КМЦ г. Читы, Поликлиническое подразделение №2",врач-терапевт участковый,2023-11-10 09:36:00+03:00,2023-11-01 02:22:25.401000+03:00,3
1,2023-11-01,3454671369,Ростовская область,1.2.643.5.1.13.13.12.2.61.6149.0.80632,"Консультативная-диагностическая поликлиника, Г...",врач-гематолог,2023-11-15 14:40:00+03:00,2023-11-01 03:47:43.915000+03:00,3
2,2023-11-01,3454726019,Приморский край,1.2.643.5.1.13.13.12.2.25.2257.0.80073,"""Владивостокская детская поликлиника №3"", ул. ...",врач-педиатр участковый,2023-11-03 10:00:00+03:00,2023-11-01 04:56:16.565000+03:00,3
3,2023-11-01,3454681503,Приморский край,1.2.643.5.1.13.13.12.2.25.2235.0.138570,"""Находкинская городская больница"", Дерматовене...",врач-дерматовенеролог,2023-11-02 02:30:00+03:00,2023-11-01 04:02:36.074000+03:00,3
4,2023-11-01,3454732666,Ханты-Мансийский автономный округ - Югра,1.2.643.5.1.13.13.12.2.86.8877.0.442366,"Женская консультация, БУ ""Нижневартовская горо...",врач-акушер-гинеколог,2023-11-15 14:10:00+03:00,2023-11-01 05:03:41.594000+03:00,3
...,...,...,...,...,...,...,...,...,...
130298,2023-11-01,3457413716,Удмуртская Республика,1.2.643.5.1.13.13.12.2.18.1322.0.72545,"Женская консультация,БУЗ УР «Глазовская МБ МЗ УР»",врач-акушер-гинеколог,2023-11-15 09:30:00+03:00,2023-11-01 22:48:53.302000+03:00,3
130299,2023-11-01,3457416780,Московская область,1.2.643.5.1.13.13.12.2.50.4716.0.546807,Поликлиническое отделение №4 (взрослое населен...,врач-терапевт,2023-11-10 16:12:00+03:00,2023-11-01 22:49:47.762000+03:00,3
130300,2023-11-01,3457447514,Белгородская область,1.2.643.5.1.13.13.12.2.31.11001.0.293035,"ОГБУЗ ""ДОКБ"" ДЕТСКАЯ ПОЛИКЛИНИКА №2",врач-педиатр участковый,2023-11-02 15:30:00+03:00,2023-11-01 22:57:38.881000+03:00,3
130301,2023-11-01,3457457469,Тульская область,1.2.643.5.1.13.13.12.2.71.7272.0.29978,"Поликлиника №2, ГУЗ ""Детская городская клиниче...",фельдшер,2023-11-02 10:40:00+03:00,2023-11-01 22:59:57.854000+03:00,3


In [51]:
df_nosub_s

,Дата,session_id,Название субъекта РФ,sp_oid,sp_name,post_name,slot_ts,create_ts,Тип записи,value
8052,2023-11-01,3455897937,Федеральное медико-биологическое агентство,1.2.643.5.1.13.13.12.2.69.7179.0.14349,"Детская поликлиника,ФБУЗ ЦМСЧ № 141 ФМБА России",врач-педиатр,2023-11-07 15:00:00+03:00,2023-11-01 13:33:25.233000+03:00,3,1
